In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# vanilla RSA adapted from seminar's RSA model
## My changes:
## Changed constructor to include truth_matrix
## Removed L0_truth_table() (now included in constructor)
## Made model more flexible, i.e. less specific to the previous assignment

## note to self: barebones for now

class RSAModel()
    def __init__(self, objects, messages, truth_matrix, alpha, priors, cost_function)
        if truth_matrix.shape != (len(objects, len(messages))):
          raise ValueError("""Truth matrix must be m x n, 
          where m is number of objects and n is the number of messages""")
        self.objects = objects
        self.messages = messages
        self.truth_matrix = truth_matrix
        self.alpha = alpha
        self.priors = priors
        self.cost_function = cost_function
        
        # the following are for convenience
        self.n_obj = len(self.objects)
        self.n_mes = len(self.messages)
    
    def __normalize(self, arr):
        return arr / arr.sum(axis=1)
    
    def L0(self):
        #compute [[m]](o): literal meaning
        literal_meaning = self.truth_table

        #P(o) -- create a matrix of priors by repeating the priors array for each row
        priors = np.tile(np.array(self.priors), (self.n_obj,1))

        #multiply the two of L0 terms together
        unnorm  =  literal_meaning * priors

        #normalize to obtain a probability distribution
        norm = self.__normalize(unnorm)

        return norm
    
    def S1(self):
        #compute the costs for each expression in self.messages
        costs = self.cost_function()

        #utility without costs
        raw_utility = np.log(self.L0().T)

        #Cost(m) -- reshape to obtain correct dimensions for matrix multiplication
        costs = np.repeat(self.cost_function(),2).reshape((self.n_mes, self.n_obj).T

        #compute utility by subtracting costs from informativity
        utility = np.subtract(raw_utility,costs)

        #compute S1(m|o) by taking the exponent of utility times the temperature parameter alpha
        unnorm = np.exp(self.alpha*utility)

        #normalize to obtain a probability distribution
        norm = self.__normalize(unnorm)

        return norm